# Historical precipitation data processing
We got the data from API in 'long' format and had to convert it to 'wide', so every fire (unique id) has a sum of rain and snow for the preceding 6 months. We got historical precipitation and snow data for only those fires with duration more than 1 day and acreage bigger than 1 acre.

In [1]:
# Imports
import pandas as pd
import numpy as np
import datetime

In [2]:
# Import of wildfires data to get the start date of the fire
df = pd.read_csv('../../data/processed/wildfire.csv')

# Fixing datetime columns format
df['FireDiscoveryDateTime'] = pd.to_datetime(df['FireDiscoveryDateTime'], infer_datetime_format=True, errors = 'coerce')
df['ControlDateTime'] = pd.to_datetime(df['ControlDateTime'], infer_datetime_format=True, errors = 'coerce')

# Filtering fires by acreage and duration to exclude 'tiny' incidents
filtered_df = (df[df['DailyAcres'] > 1])
filtered_df = filtered_df[((filtered_df['ControlDateTime'] - filtered_df['FireDiscoveryDateTime']) > datetime.timedelta(days=1))]

In [3]:
filtered_df.shape

(2863, 20)

In [4]:
# Extracting id attributes and fire start date
data_attr = filtered_df[['UniqueFireIdentifier', 'id', 'FireDiscoveryDateTime']]

In [5]:
# Processing data so we have a sum of snow and rain for the 6 previous months before the fire started
precip = pd.read_csv('../../data/raw/nasa_weather_last_180days.csv') # import of historical precipitation data gathered by POWER API
precip.head(10)

,PARAMETER,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANN,ID
0,PRECSNO,2020,1.32,0.66,0.66,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.66,0.66,0.00,1
1,PRECTOTCORR_SUM,2020,52.73,21.09,26.37,10.55,21.09,26.37,0.00,0.00,5.27,0.00,31.64,21.09,216.21,1
2,PRECSNO,2020,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5
3,PRECTOTCORR_SUM,2020,0.00,21.09,63.28,5.27,0.00,0.00,0.00,5.27,0.00,0.00,5.27,0.00,100.20,5
4,PRECSNO,2019,0.66,1.98,0.66,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.32,0.66,10
5,PRECSNO,2020,0.00,0.00,1.32,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.66,0.00,10
6,PRECTOTCORR_SUM,2019,89.65,163.48,68.55,21.09,42.19,0.00,10.55,0.00,31.64,0.00,5.27,63.28,495.70,10
7,PRECTOTCORR_SUM,2020,10.55,0.00,47.46,15.82,10.55,5.27,0.00,0.00,0.00,0.00,36.91,26.37,152.93,10
8,PRECSNO,2020,3.30,0.00,5.27,0.66,0.00,0.00,0.00,0.00,0.00,0.00,0.66,2.64,1.32,12
9,PRECSNO,2021,4.53,1.48,1.98,0.20,0.02,0.00,0.00,0.00,0.00,0.99,0.09,13.09,1.89,12


In [6]:
precip.PARAMETER.value_counts()

PRECSNO            3116
PRECTOTCORR_SUM    3116
Name: PARAMETER, dtype: int64

In [7]:
# Dropping annual column
precip = precip.drop(columns = 'ANN')

# Renaming months to their numbers
precip.columns = ['PARAMETER', 'YEAR', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', 'ID']

In [8]:
# Dividing tables to snow and rain tables
precip_rain = precip[precip.PARAMETER == 'PRECTOTCORR_SUM']
precip_snow = precip[precip.PARAMETER == 'PRECSNO']

precip_rain.shape, precip_snow.shape

((3116, 15), (3116, 15))

In [9]:
# Dropping PARAMETER columns
precip_rain = precip_rain.drop(columns = 'PARAMETER')
precip_snow = precip_snow.drop(columns = 'PARAMETER')
precip_rain.head()

,YEAR,01,02,03,04,05,06,07,08,09,10,11,12,ID
1,2020,52.73,21.09,26.37,10.55,21.09,26.37,0.00,0.00,5.27,0.0,31.64,21.09,1
3,2020,0.00,21.09,63.28,5.27,0.00,0.00,0.00,5.27,0.00,0.0,5.27,0.00,5
6,2019,89.65,163.48,68.55,21.09,42.19,0.00,10.55,0.00,31.64,0.0,5.27,63.28,10
7,2020,10.55,0.00,47.46,15.82,10.55,5.27,0.00,0.00,0.00,0.0,36.91,26.37,10
10,2020,73.83,0.00,131.84,68.55,68.55,0.00,0.00,0.00,0.00,0.0,73.83,79.10,12


In [10]:
# How many years do we have?
precip_rain.YEAR.value_counts()

2020    1792
2021     862
2019     462
Name: YEAR, dtype: int64

In [11]:
# Pivoting different years data to the same row
precip_rain_pivoted = precip_rain.pivot(index = 'ID', columns = 'YEAR')

# Renaming columns so they can be sorted
precip_rain_pivoted.columns = [(str(col[1])[-2:] + '_' + str(col[0])) for col in precip_rain_pivoted.columns]

# Sorting columns in chronological order
precip_rain_pivoted = precip_rain_pivoted.sort_index(axis=1)

precip_rain_pivoted.head(4)

,19_01,19_02,19_03,19_04,19_05,19_06,19_07,19_08,19_09,19_10,...,21_03,21_04,21_05,21_06,21_07,21_08,21_09,21_10,21_11,21_12
ID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,89.65,163.48,68.55,21.09,42.19,0.0,10.55,0.0,31.64,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,79.1,9.6,7.56,2.35,3.29,0.87,14.16,287.44,51.12,340.79


In [12]:
precip_rain_pivoted.tail(1)

,19_01,19_02,19_03,19_04,19_05,19_06,19_07,19_08,19_09,19_10,...,21_03,21_04,21_05,21_06,21_07,21_08,21_09,21_10,21_11,21_12
ID,,,,,,,,,,,,,,,,,,,,,
13165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.27,24.5,18.26,92.89,94.6,105.37,29.8,9.08,4.13,22.29


In [13]:
# Comparing number of observations
data_attr.shape

(2863, 3)

In [14]:
precip_rain_pivoted.shape

(2128, 36)

In [15]:
# Merging pivoted table with fire attributes to get the fire date
data = precip_rain_pivoted.merge(data_attr, left_on='ID', right_on = 'id')

In [16]:
data

,19_01,19_02,19_03,19_04,19_05,19_06,19_07,19_08,19_09,19_10,...,21_06,21_07,21_08,21_09,21_10,21_11,21_12,UniqueFireIdentifier,id,FireDiscoveryDateTime
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-NVECFX-010145,1,2020-07-19 23:00:00+00:00
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-AZCRD-002177,5,2020-10-15 18:17:00+00:00
2,89.65,163.48,68.55,21.09,42.19,0.0,10.55,0.0,31.64,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-NVCCD-030327,10,2020-06-24 00:17:59+00:00
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.35,3.29,0.87,14.16,287.44,51.12,340.79,2021-CATNF-000111,12,2021-01-19 19:40:00+00:00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-CAKNF-005480,15,2020-07-22 19:09:59+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.60,42.78,44.30,34.81,43.21,33.99,90.93,2021-COGRX-000596,13121,2021-12-04 22:36:00+00:00
2124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.25,53.62,37.73,24.32,26.46,21.70,59.35,2021-COCCX-000977,13122,2021-12-05 14:58:59+00:00
2125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.07,96.50,40.35,42.79,22.13,6.28,34.69,2021-NMZUA-000401,13143,2021-12-04 18:30:00+00:00
2126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,90.91,88.98,111.38,24.12,7.17,3.38,14.09,2021-NMN5S-000432,13162,2021-12-27 20:24:00+00:00


In [17]:
# Calculating year and month of fire start
data['year'] = pd.DatetimeIndex(data['FireDiscoveryDateTime']).year
data['month'] = pd.DatetimeIndex(data['FireDiscoveryDateTime']).month

In [18]:
# Creating column indicating which column to start with when calculating precipitation sum
data['year'] = data['year'].map(lambda x: str(x)[-2:]) # slicing two last digits of the year
data['month'] = data['month'].astype(str) # converting month column to string
data['month'] = data['month'].map(lambda x: ('0' + x) if len(x) == 1 else x) # adding zeros to months so they match column names
data['start'] = data['year'] + '_' + data['month'] # strings concatenation

In [19]:
# Calculating 6 months sum of precipitation preceding to the start date of the fire
sums = []
for i in range(data.shape[0]):
    col_v = data.iloc[i]['start']
    col = data.columns.get_loc(col_v)
    precip = data.iloc[i][col-6:col+1].sum()
    sums.append(precip)
# Assigning result to a new column
data['sum_rain'] = sums

In [20]:
# Repeating the sequence for snow data
precip_snow_pivoted = precip_snow.pivot(index = 'ID', columns = 'YEAR') # pivot
precip_snow_pivoted.columns = [(str(col[1])[-2:] + '_' + str(col[0])) for col in precip_snow_pivoted.columns] # renaming columns for sorting
precip_snow_pivoted = precip_snow_pivoted.sort_index(axis=1) # sorting columns chronologically

# Merging pivoted table with fire attributes to get the fire start date
data_snow = precip_snow_pivoted.merge(data_attr, left_on='ID', right_on = 'id')

# Calculating year and month of fire start
data_snow['year'] = pd.DatetimeIndex(data_snow['FireDiscoveryDateTime']).year
data_snow['month'] = pd.DatetimeIndex(data_snow['FireDiscoveryDateTime']).month

# Creating column indicating which column to start with when calculating snow sum
data_snow['year'] = data_snow['year'].map(lambda x: str(x)[-2:]) # slicing two last digits of the year
data_snow['month'] = data_snow['month'].astype(str) # converting month column to string
data_snow['month'] = data_snow['month'].map(lambda x: ('0' + x) if len(x) == 1 else x) # adding zeros to months so they match column names
data_snow['start'] = data_snow['year'] + '_' + data_snow['month'] # strings concatenation


# calculating snow sums for every id/fire location
sums = []
for i in range(data_snow.shape[0]):
    col_v = data_snow.iloc[i]['start']
    col = data_snow.columns.get_loc(col_v)
    precip = data_snow.iloc[i][col-6:col+1].sum()
    sums.append(precip)
    
# Assigning result to a new column in a summary dataset
data['sum_snow'] = sums

In [21]:
data

,19_01,19_02,19_03,19_04,19_05,19_06,19_07,19_08,19_09,19_10,...,21_11,21_12,UniqueFireIdentifier,id,FireDiscoveryDateTime,year,month,start,sum_rain,sum_snow
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2020-NVECFX-010145,1,2020-07-19 23:00:00+00:00,20,07,20_07,158.20,2.64
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2020-AZCRD-002177,5,2020-10-15 18:17:00+00:00,20,10,20_10,10.54,0.00
2,89.65,163.48,68.55,21.09,42.19,0.0,10.55,0.0,31.64,0.0,...,NaN,NaN,2020-NVCCD-030327,10,2020-06-24 00:17:59+00:00,20,06,20_06,152.93,2.64
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,51.12,340.79,2021-CATNF-000111,12,2021-01-19 19:40:00+00:00,21,01,21_01,284.77,7.83
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2020-CAKNF-005480,15,2020-07-22 19:09:59+00:00,20,07,20_07,395.50,4.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33.99,90.93,2021-COGRX-000596,13121,2021-12-04 22:36:00+00:00,21,12,21_12,317.62,2.76
2124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.70,59.35,2021-COCCX-000977,13122,2021-12-05 14:58:59+00:00,21,12,21_12,258.43,1.88
2125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.28,34.69,2021-NMZUA-000401,13143,2021-12-04 18:30:00+00:00,21,12,21_12,257.81,0.48
2126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.38,14.09,2021-NMN5S-000432,13162,2021-12-27 20:24:00+00:00,21,12,21_12,340.03,0.17


In [22]:
# Export of results so they can be merged into X 
data[['UniqueFireIdentifier', 'id', 'sum_rain','sum_snow']].to_csv('../../data/processed/meteorology_historical_rain_snow_sum_6mo.csv', index = False)